In [ ]:
import pandas as pd
import json
from sqlalchemy import create_engine
import psycopg2

# SET DATABASE CREDENTIALS
DB_USER = 'postgres'
DB_PASSWORD = 'postgress'
DB_HOST = 'localhost'
DB_PORT = '5432'
DB_NAME = 'student_database'


def extract_data(file):
    if file.endswith('.csv'):
        data = pd.read_csv(file)
        return data
    if file.endswith('.parquet'):
        data = pd.read_parquet(file)
        return data
    if file.endswith('.xlsx'):
        data = pd.read_xlsx(file)
        return data
    if file.endswith('.json'):
        with open(file, 'r') as file_obj:
            data = json.load(file_obj)
            return data
    else:
        print("Please we cannot read the file you provided\nAre you sure the file is in the right format?\nWe read only flat file(csv,parquet,json,xlsx)")

# filename = 'students.csv'
# read_data = extract_data(filename)

# TRANSFORMING THE DATA


def transform_data(data):
    data.columns = data.columns.str.lower().str.replace(' ', '_')

    data['age'] = pd.to_numeric(data['age'], errors='coerce')
    ave_age = int(data['age'].mean(skipna=True))
    data['age'].fillna(ave_age, inplace=True)

    data['registration_date'] = pd.to_datetime(
        data['registration_date'], errors='coerce').dt.date

    data.insert(0, 'student_id', range(1, len(data) + 1))
    data.dropna(subset=['name'], inplace=True)
    data.reset_index(drop=True, inplace=True)

    data.to_csv('cleaned_data.csv', index=True)
    return data


# filename = 'students.csv'
# E = extract_data(filename)
# T = transform_data(E)

# LOAD DATA
def load_data(data):
    engine = create_engine(
        f'postgress//:{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}')
    data.sql('student', engine, if_exists='replace', index=True)


if '_name' == '__main':
    file_path = 'students.csv'

    print("Starting ETL pipeline....Please wait")
    EXTRACT_DATA = extract_data(file_path)
    print("Extraction Completed...")

    print('Transforming Data...Please Wait')
    TRANSFORM_DATA = transform_data(EXTRACT_DATA)
    print('Transformation Completed..')

    print('Loading Data..Please Wait')
    load_data(TRANSFORM_DATA)
    print('Data Loaded Successfully to Postgress')